In [1]:
import pandas as pd
from datetime import datetime
from datetime import timedelta
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style
style.use('fivethirtyeight')

import plotly.express as px

import os
import sys

from scipy.optimize import minimize
# code to manage where are lunch the file
if str(os.getcwd()).split("/")[-1] == "notebooks":
    sys.path.insert(1, os.path.realpath(os.path.pardir))
    path_b = "../"
else:
    path_b ='./'
import config as c

In [2]:
windows = c.windows
number_crypto = c.number_cryptos
market_cap = c.market_cap

df_name_index = pd.read_csv(f"{path_b}data/processed/first_{number_crypto}_crypto_list.csv", index_col=0)
df = pd.read_csv(f"{path_b}data/processed/market_cap_crypto.csv", index_col=0)

df_close_price = pd.read_csv(f'{path_b}data/processed/close_price_crypto.csv', index_col=0)
df_close_price['date'] = pd.to_datetime(df_close_price.index)
df_close_price['date'] = df_close_price['date'].dt.date
df_close_price.index = df_close_price['date']
del df_close_price['date']

## use market cap file

df_market_cap = pd.read_csv(f'{path_b}data/processed/crypto_date_marketcap_sorted.csv', index_col=0)


df_market_cap_first_20 = df_market_cap.iloc[:number_crypto]

#select last date
first_date = df_market_cap_first_20['first_date_marketcap'].tail(1).values

first_date_final = pd.to_datetime(first_date[0])

df_final_data = df_close_price.drop(df_close_price[df_close_price.index < first_date_final].index)
df_final_data = df_final_data[df_market_cap_first_20['crypto_name']]
df_final_data_return = np.log(df_final_data/df_final_data.shift(1)).dropna()

/Applications/anaconda3/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:73: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  result = libops.scalar_compare(x.ravel(), y, op)


In [3]:
df_weigths = df_final_data_return.applymap(lambda x: 1/number_crypto) #
port_equal_weigthed = np.sum(np.multiply(df_final_data_return, df_weigths),1)[2:]
#bug avec ces deux dates
port_equal_weigthed = port_equal_weigthed.drop(port_equal_weigthed[port_equal_weigthed.index == pd.to_datetime("2018-05-09") ].index)
port_equal_weigthed = port_equal_weigthed.drop(port_equal_weigthed[port_equal_weigthed.index == pd.to_datetime("2018-05-10") ].index)

In [8]:
plt.figure(figsize=(15,7))
plt.plot((port_equal_weigthed+1).cumprod()*100)
plt.legend(['Equally weigthed Index'],loc='upper left',fontsize='large',fancybox=True, shadow=True)
plt.title('Performance of Equally weigthed Index')
plt.show()

EW = port_equal_weigthed+1).cumprod()*100
EW.to_csv(f"../data/processed/EW_{number_crypto}_price.csv")

SyntaxError: unmatched ')' (<ipython-input-8-a334319f33b0>, line 7)

# Debug pour les dates

In [5]:
df_test = pd.DataFrame((port_equal_weigthed+1).cumprod())
df_test.columns = ['cum_perf']
# df_test
df_test.drop(df_test[df_test.cum_perf == -np.inf ].index)

,cum_perf
date,
2019-02-02,1.013956
2019-02-03,0.987440
2019-02-04,0.996789
2019-02-05,1.002991
2019-02-06,0.974366
...,...
2021-10-13,3.607941
2021-10-14,3.637484
2021-10-15,3.690066


In [6]:
port_equal_weigthed.drop(port_equal_weigthed[port_equal_weigthed.index < pd.to_datetime("2018-05-06") ].index)

/Applications/anaconda3/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:73: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  result = libops.scalar_compare(x.ravel(), y, op)


date
2019-02-02    0.013956
2019-02-03   -0.026151
2019-02-04    0.009467
2019-02-05    0.006223
2019-02-06   -0.028540
                ...   
2021-10-13    0.031585
2021-10-14    0.008188
2021-10-15    0.014456
2021-10-16   -0.001040
2021-10-17    0.000919
Length: 989, dtype: float64

In [7]:
(port_equal_weigthed+1).cumprod()

date
2019-02-02    1.013956
2019-02-03    0.987440
2019-02-04    0.996789
2019-02-05    1.002991
2019-02-06    0.974366
                ...   
2021-10-13    3.607941
2021-10-14    3.637484
2021-10-15    3.690066
2021-10-16    3.686230
2021-10-17    3.689616
Length: 989, dtype: float64